<a href="https://colab.research.google.com/github/elephanti/NLPProject2024/blob/main/LAMBADA%20Data%20Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations & imports

In [1]:
!pip install bitsandbytes
!pip install transformers
!pip install accelerate
!pip install peft
!pip install datasets
!pip install evaluate
!pip install flash-attn --no-build-isolation
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from huggingface_hub import notebook_login
from datasets import Dataset
import pandas as pd
import os
from DataAugmenter import DataAugmenter
import gc
import torch

# HuggingFace Login

In [3]:
notebook_login()

# Datasets loading & preprocessing

In [6]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_ATIS_VER_1_10",
    model_name = "mistral",
    quantize_4bit=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv -P /content/datasets/ATIS/VER1
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/ATIS/sampled_subsets/ver1/atis_10_subset.csv -P /content/datasets/ATIS/VER1
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/ATIS/sampled_subsets/ver1/atis_20_subset.csv -P /content/datasets/ATIS/VER1
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/ATIS/sampled_subsets/ver1/atis_50_subset.csv -P /content/datasets/ATIS/VER1
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/ATIS/sampled_subsets/ver1/atis_100_subset.csv -P /content/datasets/ATIS/VER1

--2024-07-18 16:17:36--  https://github.com/elephanti/NLPProject2024/raw/main/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/elephanti/NLPProject2024/main/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv [following]
--2024-07-18 16:17:36--  https://raw.githubusercontent.com/elephanti/NLPProject2024/main/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5483 (5.4K) [text/plain]
Saving to: ‘/content/datasets/ATIS/VER1/atis_5_subset.csv.1’

atis_5_subset.csv.1 100%[===================>]   5.

In [7]:
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv -P /content/datasets/TREC/VER1
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/TREC/sampled_subsets/ver1/trec_10_subset.csv -P /content/datasets/TREC/VER1
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/TREC/sampled_subsets/ver1/trec_20_subset.csv -P /content/datasets/TREC/VER1
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/TREC/sampled_subsets/ver1/trec_50_subset.csv -P /content/datasets/TREC/VER1
!wget https://github.com/elephanti/NLPProject2024/raw/main/datasets/TREC/sampled_subsets/ver1/trec_100_subset.csv -P /content/datasets/TREC/VER1

--2024-07-19 07:55:50--  https://github.com/elephanti/NLPProject2024/raw/main/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/elephanti/NLPProject2024/main/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv [following]
--2024-07-19 07:55:50--  https://raw.githubusercontent.com/elephanti/NLPProject2024/main/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32861 (32K) [text/plain]
Saving to: ‘/content/datasets/TREC/VER1/trec_5_subset.csv’

trec_5_subset.csv   100%[===================>]  32.09

In [ ]:
atis_train_5_dataset = data_augmenter.load_and_preprocess_dataset('datasets/ATIS/VER1/atis_5_subset.csv')
atis_train_10_dataset = data_augmenter.load_and_preprocess_dataset('datasets/ATIS/VER1/atis_10_subset.csv')
atis_train_20_dataset = data_augmenter.load_and_preprocess_dataset('datasets/ATIS/VER1/atis_20_subset.csv')
atis_train_50_dataset = data_augmenter.load_and_preprocess_dataset('datasets/ATIS/VER1/atis_50_subset.csv')
atis_train_100_dataset = data_augmenter.load_and_preprocess_dataset('datasets/ATIS/VER1/atis_100_subset.csv')

In [8]:
trec_train_5_dataset = data_augmenter.load_and_preprocess_dataset('datasets/TREC/VER1/trec_5_subset.csv')
trec_train_10_dataset = data_augmenter.load_and_preprocess_dataset('datasets/TREC/VER1/trec_10_subset.csv')
trec_train_20_dataset = data_augmenter.load_and_preprocess_dataset('datasets/TREC/VER1/trec_20_subset.csv')
trec_train_50_dataset = data_augmenter.load_and_preprocess_dataset('datasets/TREC/VER1/trec_50_subset.csv')
trec_train_100_dataset = data_augmenter.load_and_preprocess_dataset('datasets/TREC/VER1/trec_100_subset.csv')

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transfor

In [ ]:
atis_intents = atis_train_5_dataset.to_pandas()['label'].unique()
atis_intents

array(['ground_fare', 'flight_time', 'capacity', 'restriction', 'city',
       'meal', 'aircraft', 'airfare', 'flight_no', 'abbreviation',
       'quantity', 'airline', 'flight', 'airport', 'day_name', 'distance',
       'ground_service', 'cheapest'], dtype=object)

In [9]:
trec_intents = trec_train_5_dataset.to_pandas()['label'].unique()
trec_intents

array(['ENTY:product', 'DESC:reason', 'NUM:weight', 'HUM:ind',
       'ENTY:substance', 'HUM:desc', 'ENTY:word', 'NUM:code', 'HUM:gr',
       'ENTY:symbol', 'ABBR:abb', 'ENTY:dismed', 'ABBR:exp',
       'ENTY:termeq', 'LOC:other', 'NUM:volsize', 'ENTY:body',
       'ENTY:color', 'LOC:country', 'DESC:def', 'ENTY:veh', 'NUM:ord',
       'LOC:state', 'NUM:date', 'ENTY:cremat', 'ENTY:food', 'NUM:temp',
       'NUM:count', 'NUM:speed', 'ENTY:other', 'HUM:title', 'ENTY:sport',
       'ENTY:letter', 'NUM:period', 'LOC:mount', 'ENTY:event',
       'ENTY:plant', 'ENTY:animal', 'NUM:perc', 'ENTY:religion',
       'DESC:desc', 'NUM:dist', 'ENTY:instru', 'LOC:city', 'NUM:money',
       'DESC:manner', 'ENTY:currency', 'NUM:other', 'ENTY:lang',
       'ENTY:techmeth'], dtype=object)

# Genereration without fine tuning

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_raw_ATIS",
    model_name = "mistral",
    quantize_4bit=True
)

generated_samples = data_augmenter.augment(atis_intents[:3], 3)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AttributeError: 'NoneType' object has no attribute 'head'

In [10]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_raw_TREC",
    model_name = "mistral",
    quantize_4bit=True
)

generated_samples = data_augmenter.augment(trec_intents[:3], 3)
print(generated_samples.head(5))

data_augmenter.clean()
del data_augmenter

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


          label                                               text
0  ENTY:product  Developed by the ENTy Lab (2015) in collaborat...
1  ENTY:product  Ep. 47 – In Conversation With ENTITY01, Ajay S...
2  ENTY:product  The latest iteration of the AX7091, a 4-core e...
3   DESC:reason  4\n\tjmp $130d97 ; RSXM_REASON_NOTFOUND\n.db "...
4   DESC:reason  === C++ 2019 ===\nThe following section lists ...


# Finetune model & generate

## ATIS

### ATIS Ver1 - 5 samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_ATIS_5",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(atis_train_5_dataset, logging_steps=10)

# Generate
generated_samples = data_augmenter.augment(atis_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/83 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,4.925000
20,3.875000
30,3.125000
40,2.912500
50,2.500000
60,2.450000
70,2.212500
80,2.075000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/185M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 986.00 MiB. GPU 

### ATIS Ver1 - 10 samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_ATIS_10",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(atis_train_10_dataset, logging_steps=20)

# Generate
generated_samples = data_augmenter.augment(atis_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

### ATIS Ver1 - 20 samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_ATIS_20",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(atis_train_20_dataset, logging_steps=20)

# Generate
generated_samples = data_augmenter.augment(atis_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

### ATIS Ver1 - 50 samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_ATIS_50",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(atis_train_50_dataset, num_train_epochs=10, save_steps=200, logging_steps=50)

# Generate
generated_samples = data_augmenter.augment(atis_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

### ATIS Ver1 - 100 samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_ATIS_100",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(atis_train_100_dataset, num_train_epochs=10, save_steps=200, logging_steps=50)

# Generate
generated_samples = data_augmenter.augment(atis_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

## TREC

### TREC Ver1 - 5 samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_TREC_5",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(trec_train_5_dataset, logging_steps=10)

# Generate
generated_samples = data_augmenter.augment(trec_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### TREC Ver1 - 10 samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_TREC_10",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(trec_train_10_dataset, logging_steps=20)

# Generate
generated_samples = data_augmenter.augment(trec_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

### TREC Ver1 - 20 Samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_TREC_20",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(trec_train_20_dataset, logging_steps=20)

# Generate
generated_samples = data_augmenter.augment(trec_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

### TREC Ver1 - 50 Samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_TREC_50",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(trec_train_50_dataset, num_train_epochs=10, save_steps=200, logging_steps=50)

# Generate
generated_samples = data_augmenter.augment(trec_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

### TREC Ver1 - 100 samples

In [ ]:
data_augmenter = DataAugmenter(
    experiment_name = "Mistral_7B_4bit_tuned_TREC_100",
    model_name = "mistral",
    quantize_4bit=True
)

# Train
data_augmenter.train(trec_train_100_dataset, num_train_epochs=10, save_steps=200, logging_steps=50)

# Generate
generated_samples = data_augmenter.augment(trec_intents, 100)
# print(generated_samples.head(20))

data_augmenter.clean()
del data_augmenter

# Download generated data

In [ ]:
from google.colab import files
files.download("/content/generated_data/Mistral_7B_4bit_raw_ATIS_augmented_data.csv")
files.download("/content/generated_data/Mistral_7B_4bit_tuned_ATIS_100_augmented_data.csv")
files.download("/content/generated_data/Mistral_7B_4bit_tuned_ATIS_10_augmented_data.csv")
files.download("/content/generated_data/Mistral_7B_4bit_tuned_ATIS_20_augmented_data.csv")
files.download("/content/generated_data/Mistral_7B_4bit_tuned_ATIS_5_augmented_data.csv")
files.download("/content/generated_data/Mistral_7B_4bit_tuned_ATIS_50_augmented_data.csv")